# Imports

In [ ]:
import logging
import random
from abc import abstractmethod
from enum import Enum
from typing import List, Optional, Type, Union

In [ ]:
import gymnasium as gym
import numpy as np
import pandas as pd
from gymnasium import spaces
from gymnasium.utils import seeding
from pandas import DataFrame

from freqtrade.exceptions import OperationalException


logger = logging.getLogger(__name__)

# Actions

In [ ]:
class BaseActions(Enum):
    """
    Default action space, mostly used for type handling.
    """

    Neutral = 0
    Long_enter = 1
    Long_exit = 2
    Short_enter = 3
    Short_exit = 4

# Positions

In [ ]:
class Positions(Enum):
    Short = 0
    Long = 1
    Neutral = 0.5

    def opposite(self):
        return Positions.Short if self == Positions.Long else Positions.Long

# Environment

## Init

In [ ]:
class BaseEnvironment(gym.Env):
    """
    Base class for environments. This class is agnostic to action count.
    Inherited classes customize this to include varying action counts/types,
    See RL/Base5ActionRLEnv.py and RL/Base4ActionRLEnv.py
    """

    def __init__(
        self,
        df: DataFrame = DataFrame(),
        prices: DataFrame = DataFrame(),
        reward_kwargs: dict = {},
        window_size=10,
        starting_point=True,
        id: str = "baseenv-1",
        seed: int = 1,
        config: dict = {},
        live: bool = False,
        fee: float = 0.0015,
        can_short: bool = False,
        pair: str = "",
        df_raw: DataFrame = DataFrame(),
    ):
        """
        Initializes the training/eval environment.
        :param df: dataframe of features
        :param prices: dataframe of prices to be used in the training environment
        :param window_size: size of window (temporal) to pass to the agent
        :param reward_kwargs: extra config settings assigned by user in `rl_config`
        :param starting_point: start at edge of window or not
        :param id: string id of the environment (used in backend for multiprocessed env)
        :param seed: Sets the seed of the environment higher in the gym.Env object
        :param config: Typical user configuration file
        :param live: Whether or not this environment is active in dry/live/backtesting
        :param fee: The fee to use for environmental interactions.
        :param can_short: Whether or not the environment can short
        """
        self.config: dict = config
        self.rl_config: dict = config["freqai"]["rl_config"]
        self.add_state_info: bool = self.rl_config.get("add_state_info", False)
        self.id: str = id
        self.max_drawdown: float = 1 - \
            self.rl_config.get("max_training_drawdown_pct", 0.8)
        self.compound_trades: bool = config["stake_amount"] == "unlimited"
        self.pair: str = pair
        self.raw_features: DataFrame = df_raw
        if self.config.get("fee", None) is not None:
            self.fee = self.config["fee"]
        else:
            self.fee = fee

        # set here to default 5Ac, but all children envs can override this
        self.actions: Type[Enum] = BaseActions  # BaseActions é 5Ac
        self.tensorboard_metrics: dict = {}
        self.can_short: bool = can_short
        self.live: bool = live
        if not self.live and self.add_state_info:
            raise OperationalException(
                "`add_state_info` is not available in backtesting. Change "
                "parameter to false in your rl_config. See `add_state_info` "
                "docs for more info."
            )
        self.seed(seed)
        self.reset_env(df, prices, window_size, reward_kwargs, starting_point)

## Reset Env

In [ ]:
    def reset_env(
        self,
        df: DataFrame,
        prices: DataFrame,
        window_size: int,
        reward_kwargs: dict,
        starting_point=True,
    ):
        """
        Resets the environment when the agent fails (in our case, if the drawdown
        exceeds the user set max_training_drawdown_pct)
        :param df: dataframe of features
        :param prices: dataframe of prices to be used in the training environment
        :param window_size: size of window (temporal) to pass to the agent
        :param reward_kwargs: extra config settings assigned by user in `rl_config`
        :param starting_point: start at edge of window or not
        """
        self.signal_features: DataFrame = df
        self.prices: DataFrame = prices
        self.window_size: int = window_size
        self.starting_point: bool = starting_point
        self.rr: float = reward_kwargs["rr"]
        self.profit_aim: float = reward_kwargs["profit_aim"]

        # # spaces
        if self.add_state_info:
            self.total_features = self.signal_features.shape[1] + 3
        else:
            self.total_features = self.signal_features.shape[1]
        self.shape = (window_size, self.total_features)
        self.set_action_space()
        self.observation_space = spaces.Box(
            low=-1, high=1, shape=self.shape, dtype=np.float32)

        # episode
        self._start_tick: int = self.window_size
        self._end_tick: int = len(self.prices) - 1
        self._done: bool = False
        self._current_tick: int = self._start_tick
        self._last_trade_tick: Optional[int] = None
        self._position = Positions.Neutral
        self._position_history: list = [None]
        self.total_reward: float = 0
        self._total_profit: float = 1
        self._total_unrealized_profit: float = 1
        self.history: dict = {}
        self.trade_history: list = []

In [ ]:
    def get_attr(self, attr: str):
        """
        Returns the attribute of the environment
        :param attr: attribute to return
        :return: attribute
        """
        return getattr(self, attr)

Esse código define um método reset_env para redefinir o ambiente de uma simulação, geralmente utilizado em algoritmos de aprendizado por reforço (Reinforcement Learning, RL) no contexto de negociação financeira. Esse método é projetado para resetar variáveis e parâmetros do ambiente quando o agente falha ou atinge uma condição de parada, como um drawdown (perda máxima) excessivo. Vamos entender cada parte:

Parâmetros do método
df: DataFrame: DataFrame com as features (ou variáveis de entrada) que o agente utilizará para tomar decisões.
prices: DataFrame: DataFrame contendo os preços dos ativos no ambiente de treino, importante para calcular retornos, lucros, etc.
window_size: int: Tamanho da janela temporal usada para apresentar dados ao agente. Um valor de 10, por exemplo, significa que o agente observa 10 períodos anteriores a cada ação.
reward_kwargs: dict: Dicionário de configurações adicionais para o cálculo de recompensa (parâmetros personalizados passados pelo usuário).
starting_point: bool: Define se o ponto de início da janela de observação está na borda do conjunto de dados (True).
Corpo do Método
Definição de variáveis e parâmetros iniciais:

self.signal_features: Atribui as features ao ambiente.
self.prices: Define os preços para o ambiente.
self.window_size: Armazena o tamanho da janela temporal.
self.starting_point: Define se deve começar na borda ou não.
self.rr e self.profit_aim: Extraem valores de recompensa do dicionário reward_kwargs.
Definição dos espaços:

self.total_features: Total de features considerando informações adicionais de estado, caso self.add_state_info esteja ativado (como features extras para o estado do agente).
self.shape: Forma da janela observada pelo agente, que depende de window_size e do total de features.
self.set_action_space(): Chama um método (presumivelmente) para configurar o espaço de ações, que define quais ações o agente pode tomar.
self.observation_space: Define o espaço de observação (ou espaço de estados), com limites entre -1 e 1, baseado em window_size e total_features.
Inicialização do episódio:

Define variáveis para gerenciar o estado do episódio:
self._start_tick: O primeiro tick observável com base em window_size.
self._end_tick: O último tick observável, baseado no tamanho de prices.
self._done: Variável booleana para sinalizar o término do episódio.
self._current_tick: Marca o tick atual no episódio.
self._last_trade_tick: Guarda o tick da última transação (inicialmente None).
self._position: Define a posição inicial do agente como Neutral.
self._position_history: Histórico das posições do agente.
self.total_reward: Recompensa acumulada durante o episódio.
self._total_profit e self._total_unrealized_profit: Lucro total e lucro não realizado acumulados.
self.history: Dicionário para guardar o histórico completo do episódio.
self.trade_history: Lista para guardar o histórico de transações.
Esse método é essencial para reinicializar o ambiente para que o agente possa começar um novo episódio de treino com condições iniciais específicas, mantendo o ambiente consistente e preparado para realizar novas decisões de negociação.

In [ ]:
@abstractmethod
    def set_action_space(self):
        """
        Unique to the environment action count. Must be inherited.
        """

O decorador @abstractmethod indica que set_action_space é um método abstrato, ou seja, ele precisa ser implementado em qualquer classe que herde da classe onde esse método está definido.

Detalhamento
@abstractmethod: Esse decorador é usado para marcar métodos em uma classe abstrata (normalmente uma classe que herda de ABC do módulo abc). Classes abstratas servem como modelos para outras classes. Elas não podem ser instanciadas diretamente e contêm métodos que devem ser obrigatoriamente implementados pelas classes filhas.

Método set_action_space: Este método define o espaço de ação para o ambiente. Em um contexto de aprendizado por reforço, o espaço de ação representa todas as ações possíveis que o agente pode realizar no ambiente. Cada ambiente de aprendizado por reforço pode ter um conjunto específico de ações (por exemplo, comprar, vender, manter posição, etc.), e set_action_space é onde esse conjunto é configurado.

Must be inherited: A docstring ("""Unique to the environment action count. Must be inherited.""") indica que o método set_action_space deve ser implementado especificamente em cada ambiente que herda desta classe. Isso permite que cada ambiente tenha sua própria contagem e tipo de ações, adaptadas às suas necessidades particulares.

Exemplo de Uso
Ao implementar uma classe filha que herda dessa classe base, você deve definir set_action_space. Por exemplo:

python
Copy code
class TradingEnv(CustomEnvBase):
    def set_action_space(self):
        # Define um espaço de ação específico para o ambiente de negociação
        self.action_space = spaces.Discrete(3)  # Exemplo: 3 ações possíveis (comprar, vender, manter)
Neste exemplo, a implementação do método set_action_space na classe TradingEnv define o espaço de ações como um espaço discreto com 3 ações.

In [ ]:
    def action_masks(self) -> List[bool]:
        return [self._is_valid(action.value) for action in self.actions]

Esse método action_masks retorna uma lista de valores booleanos, onde cada valor indica se uma ação é válida ou não no momento atual. Vamos entender cada parte:

Análise do Método action_masks
Retorno do Método:

O método retorna uma lista de valores booleanos (List[bool]), onde cada elemento da lista é o resultado da verificação de uma ação específica.
Lista de Ações:

O método percorre self.actions, que parece ser uma lista de ações possíveis no ambiente. Cada ação provavelmente é uma instância de uma classe que tem um atributo value, representando o valor específico ou o identificador da ação.
Validação das Ações com _is_valid:

Para cada action em self.actions, o método chama self._is_valid(action.value).
Esse _is_valid é presumivelmente um método auxiliar que verifica se uma determinada ação (identificada por action.value) é permitida ou válida no estado atual do ambiente.
A implementação de _is_valid não está visível aqui, mas é provável que ele contenha alguma lógica condicional baseada no estado do ambiente ou em restrições específicas.
Exemplo de Uso:

Esse método é frequentemente utilizado em ambientes de aprendizado por reforço onde algumas ações podem não estar disponíveis ou recomendadas em certos estados.
Ao retornar uma lista de máscaras de ações (True para ações válidas e False para inválidas), o ambiente pode restringir o agente a realizar apenas ações que fazem sentido em um determinado estado.
Exemplo Prático
Suponha que estamos em um ambiente de negociação, onde o agente pode comprar, vender ou manter posição. Em algumas situações, algumas ações podem não ser permitidas:

Se o agente já comprou e está com uma posição aberta, a ação de comprar novamente pode ser inválida.
Da mesma forma, vender sem ter uma posição aberta também seria inválido.
Nesse caso, o método action_masks pode retornar algo como:

python
Copy code
[True, False, True]  # Onde "True" representa uma ação válida e "False" uma inválida.
Resumo
O método action_masks fornece uma forma eficiente de informar ao agente quais ações são viáveis em um determinado momento, evitando que ele realize ações inválidas. Ele é útil para algoritmos de aprendizado por reforço que precisam filtrar ações disponíveis em tempo real.

In [ ]:
    def seed(self, seed: int = 1):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

In [ ]:
def tensorboard_log(
        self,
        metric: str,
        value: Optional[Union[int, float]] = None,
        inc: Optional[bool] = None,
        category: str = "custom",
    ):
        """
        Function builds the tensorboard_metrics dictionary
        to be parsed by the TensorboardCallback. This
        function is designed for tracking incremented objects,
        events, actions inside the training environment.
        For example, a user can call this to track the
        frequency of occurrence of an `is_valid` call in
        their `calculate_reward()`:

        def calculate_reward(self, action: int) -> float:
            if not self._is_valid(action):
                self.tensorboard_log("invalid")
                return -2

        :param metric: metric to be tracked and incremented
        :param value: `metric` value
        :param inc: (deprecated) sets whether the `value` is incremented or not
        :param category: `metric` category
        """
        increment = True if value is None else False
        value = 1 if increment else value

        if category not in self.tensorboard_metrics:
            self.tensorboard_metrics[category] = {}

        if not increment or metric not in self.tensorboard_metrics[category]:
            self.tensorboard_metrics[category][metric] = value
        else:
            self.tensorboard_metrics[category][metric] += value

    def reset_tensorboard_log(self):
        self.tensorboard_metrics = {}

In [ ]:
def reset(self, seed=None):
    """
        Reset is called at the beginning of every episode
        """
     self.reset_tensorboard_log()

      self._done = False

       if self.starting_point is True:
            if self.rl_config.get("randomize_starting_position", False):
                length_of_data = int(self._end_tick / 4)
                start_tick = random.randint(
                    self.window_size + 1, length_of_data)
                self._start_tick = start_tick
            self._position_history = (
                self._start_tick * [None]) + [self._position]
        else:
            self._position_history = (
                self.window_size * [None]) + [self._position]

        self._current_tick = self._start_tick
        self._last_trade_tick = None
        self._position = Positions.Neutral

        self.total_reward = 0.0
        self._total_profit = 1.0  # unit
        self.history = {}
        self.trade_history = []
        self.portfolio_log_returns = np.zeros(len(self.prices))

        self._profits = [(self._start_tick, 1)]
        self.close_trade_profit = []
        self._total_unrealized_profit = 1

        return self._get_observation(), self.history

Esse método reset redefine o ambiente no início de cada episódio, restaurando variáveis e estados para iniciar uma nova sequência de interações. Em aprendizado por reforço, isso é essencial para garantir que cada episódio comece com uma condição inicial consistente ou aleatorizada, conforme necessário. Vamos detalhar cada parte do código:

Estrutura do Método reset
self.reset_tensorboard_log():

Este método parece redefinir os logs no TensorBoard (uma ferramenta de visualização para o treinamento de redes neurais), garantindo que os dados de logs do episódio anterior não interfiram no novo episódio.
Definição do Estado Inicial:

self._done = False: Define que o episódio não está concluído (False). Esta variável será utilizada para indicar quando o episódio termina.
Inicialização do start_tick:

self.starting_point: Dependendo do valor de self.starting_point, a posição inicial (_start_tick) pode ser escolhida de duas formas:
Se self.rl_config.get("randomize_starting_position", False) for True, o início é aleatório dentro de um intervalo que evita o começo exato dos dados. Isso ajuda o agente a treinar em diferentes partes da série temporal.
Caso contrário, define-se uma posição de início fixa, ou no início do intervalo (window_size), se starting_point for False.
Histórico de Posições:

self._position_history: Inicializa o histórico de posições, registrando o estado inicial como None até o _start_tick. Isso é útil para acompanhar as posições do agente ao longo do episódio.
Outras Variáveis de Estado:

self._current_tick: Define o tick atual como _start_tick, indicando o ponto de início do episódio.
self._last_trade_tick = None: Inicializa o último tick de negociação como None, pois ainda não houve negociações no início do episódio.
self._position = Positions.Neutral: Define a posição inicial do agente como Neutral, indicando que ele não possui uma posição de negociação ativa no início.
Inicialização de Métricas e Logs:

self.total_reward = 0.0: Define a recompensa total acumulada no episódio como 0.0.
self._total_profit = 1.0: Inicializa o lucro total como 1.0 (provavelmente um valor neutro, onde um lucro multiplicativo aumentaria esse valor).
self.history = {} e self.trade_history = []: Define dicionários ou listas vazias para armazenar o histórico de estados e transações durante o episódio.
self.portfolio_log_returns = np.zeros(len(self.prices)): Cria um array de zeros do tamanho de prices, que armazenará os retornos logarítmicos do portfólio ao longo do tempo.
Lucros e Negociações Fechadas:

self._profits = [(self._start_tick, 1)]: Inicializa uma lista de lucros onde o primeiro valor é o start_tick e 1, indicando um lucro inicial neutro.
self.close_trade_profit = []: Lista para armazenar os lucros de negociações fechadas.
self._total_unrealized_profit = 1: Define o lucro não realizado como 1, representando o ponto de partida sem lucros ou perdas acumuladas.
Retorno da Observação Inicial e Histórico:

return self._get_observation(), self.history: Retorna a primeira observação (self._get_observation()) e o histórico inicial vazio. A primeira observação será o estado que o agente usará como entrada para decidir sua primeira ação.
Resumo
Esse método prepara o ambiente para um novo episódio, reiniciando todas as variáveis críticas, incluindo o histórico de transações, lucros, recompensas acumuladas e posição atual. Isso garante que cada episódio comece em um estado limpo e consistente, o que é essencial para o aprendizado por reforço, especialmente ao treinar e avaliar o desempenho do agente em diferentes condições do ambiente.

As diferenças entre os métodos reset_env e reset estão na função e na finalidade de cada um no contexto de um ambiente de aprendizado por reforço. Vamos detalhar essas diferenças:

1. Propósito Geral
reset_env: Esse método é voltado para redefinir o ambiente em termos de configuração inicial e variáveis globais que controlam a simulação. Ele configura as variáveis essenciais do ambiente, como dados de preços, recursos de sinal, tamanho da janela, e parâmetros de recompensa. Esse método é mais sobre preparar o ambiente de forma completa e é chamado antes de qualquer episódio ser iniciado.

reset: Esse método é chamado no início de cada episódio e redefine apenas as variáveis necessárias para o episódio atual. Ele prepara o ambiente para o começo do episódio, reiniciando variáveis como o tick inicial, histórico de posições, total de lucros e recompensas acumuladas. O reset assume que o ambiente já foi configurado pelo reset_env e apenas coloca o agente no ponto de partida do episódio.

2. Configuração do Ambiente vs. Preparação para o Episódio
reset_env: É responsável por configurar o ambiente inteiro (por exemplo, os dados de entrada, espaço de observação, espaço de ação). Pode incluir:
Definir o espaço de observação e de ação (exemplo: self.observation_space e self.action_space).
Configurar variáveis globais que não mudam a cada episódio, como dados históricos de preços ou variáveis de configuração (como reward_kwargs).
Esse método pode ser chamado uma vez por simulação ou em casos específicos onde há necessidade de uma reconfiguração completa do ambiente.
reset: Prepara o ambiente apenas para o episódio atual. Isso inclui:
Inicializar ou reiniciar variáveis de estado e métricas do episódio, como total_reward, total_profit, e position_history.
Definir a posição inicial do agente (por exemplo, start_tick).
Este método é chamado no início de cada episódio, garantindo que o agente comece de um estado limpo e consistente para cada nova execução.
3. Exemplo de Fluxo de Uso
Em um fluxo de treinamento típico:

reset_env é chamado para configurar o ambiente com dados e variáveis globais. Isso ocorre apenas uma vez antes do início dos episódios, ou quando há necessidade de uma reconfiguração completa.

Para cada novo episódio:

O método reset é chamado para reinicializar as variáveis específicas do episódio, como a posição do agente, as recompensas acumuladas, e os lucros iniciais.
O episódio então é executado, com o agente realizando ações e o ambiente respondendo a essas ações.
Ao final do episódio, o ambiente pode chamar reset novamente para preparar o próximo episódio.
Resumo
reset_env: Configuração inicial e global do ambiente, chamada uma vez por simulação ou quando o ambiente precisa ser redefinido por completo.
reset: Preparação específica para cada episódio, garantindo que o agente comece cada um com um estado limpo e as variáveis do episódio reinicializadas.
Essa divisão ajuda a manter o código organizado e eficiente, pois reset_env lida com a configuração mais pesada e reset apenas prepara o ambiente para um novo episódio sem reconfigurar tudo.

In [ ]:
@abstractmethod
    def step(self, action: int):
        """
        Step depends on action types, this must be inherited.
        """
        return

In [ ]:
    def _get_observation(self):
        """
        This may or may not be independent of action types, user can inherit
        this in their custom "MyRLEnv"
        """
        features_window = self.signal_features[
            (self._current_tick - self.window_size): self._current_tick
        ]
        if self.add_state_info:
            features_and_state = DataFrame(
                np.zeros((len(features_window), 3)),
                columns=["current_profit_pct", "position", "trade_duration"],
                index=features_window.index,
            )

            features_and_state["current_profit_pct"] = self.get_unrealized_profit(
            )
            features_and_state["position"] = self._position.value
            features_and_state["trade_duration"] = self.get_trade_duration()
            features_and_state = pd.concat(
                [features_window, features_and_state], axis=1)
            return features_and_state
        else:
            return features_window

O método _get_observation é responsável por construir e retornar a observação atual do ambiente, ou seja, o estado que o agente "vê" no momento. Esse método inclui um conjunto de variáveis (ou "features") sobre o ambiente e, opcionalmente, algumas informações adicionais de estado.

Detalhamento do Método
Construção da Janela de Features:

features_window: Este trecho seleciona uma janela de dados das signal_features (variáveis de entrada, como indicadores de mercado). A janela tem tamanho self.window_size e termina no self._current_tick, ou seja, o ponto atual no episódio.
Seleção da Janela: Esse recorte (self._current_tick - self.window_size): self._current_tick gera uma sequência dos dados mais recentes que o agente usa como contexto.
Adição de Informações de Estado (Condicional):

self.add_state_info: Se essa variável estiver definida como True, o método adiciona informações de estado adicionais ao conjunto de observação, como:
current_profit_pct: Calcula o lucro não realizado atual usando o método get_unrealized_profit().
position: Indica a posição atual do agente (por exemplo, comprada, vendida ou neutra), acessada através de self._position.value.
trade_duration: Calcula a duração da negociação atual (ou seja, o número de ticks desde que a posição foi aberta) usando get_trade_duration().
Essas informações são adicionadas a um DataFrame features_and_state, que é então concatenado com features_window para criar uma visão mais ampla do estado.
Concatenação de Dados:

O features_and_state é criado como um novo DataFrame que contém as três colunas adicionais: current_profit_pct, position, e trade_duration. Ele é concatenado com features_window ao longo do eixo das colunas (axis=1), formando um único DataFrame com todas as features, incluindo as informações adicionais de estado.
Retorno da Observação:

Quando add_state_info é True: Retorna o DataFrame concatenado com as features e as informações de estado.
Quando add_state_info é False: Retorna apenas features_window, sem as informações adicionais.
Finalidade do Método
Esse método é configurável para fornecer ao agente observações personalizadas. Em algumas aplicações de aprendizado por reforço, informações adicionais de estado podem ser úteis para aumentar o contexto que o agente usa para decidir ações, enquanto em outras, manter o estado simples (apenas as features de entrada) é preferível. Essa flexibilidade permite que o ambiente seja adaptado para diferentes tipos de agentes e experimentos de RL.

Exemplo de Uso
Se o window_size for 10, e o current_tick for 15, a observação retornada incluirá os dados dos ticks 5 a 15 nas signal_features, com ou sem as informações adicionais de estado, dependendo da configuração.

Esse design permite que o método _get_observation seja herdado e modificado em uma classe customizada (MyRLEnv), dando ao usuário a possibilidade de ajustar o tipo de observação que o agente recebe.

In [ ]:
    def get_trade_duration(self):
        """
        Get the trade duration if the agent is in a trade
        """
        if self._last_trade_tick is None:
            return 0
        else:
            return self._current_tick - self._last_trade_tick

In [ ]:

    def get_unrealized_profit(self):
        """
        Get the unrealized profit if the agent is in a trade
        """
        if self._last_trade_tick is None:
            return 0.0

        if self._position == Positions.Neutral:
            return 0.0
        elif self._position == Positions.Short:
            current_price = self.add_entry_fee(
                self.prices.iloc[self._current_tick].open)
            last_trade_price = self.add_exit_fee(
                self.prices.iloc[self._last_trade_tick].open)
            return (last_trade_price - current_price) / last_trade_price
        elif self._position == Positions.Long:
            current_price = self.add_exit_fee(
                self.prices.iloc[self._current_tick].open)
            last_trade_price = self.add_entry_fee(
                self.prices.iloc[self._last_trade_tick].open)
            return (current_price - last_trade_price) / last_trade_price
        else:
            return 0.0


In [ ]:

    @abstractmethod
    def is_tradesignal(self, action: int) -> bool:
        """
        Determine if the signal is a trade signal. This is
        unique to the actions in the environment, and therefore must be
        inherited.
        """
        return True


In [ ]:
    def _is_valid(self, action: int) -> bool:
        """
        Determine if the signal is valid.This is
        unique to the actions in the environment, and therefore must be
        inherited.
        """
        return True

In [ ]:
    def add_entry_fee(self, price):
        return price * (1 + self.fee)

    def add_exit_fee(self, price):
        return price / (1 + self.fee)

In [ ]:
    def _update_history(self, info):
        if not self.history:
            self.history = {key: [] for key in info.keys()}

        for key, value in info.items():
            self.history[key].append(value)

In [ ]:
    @abstractmethod
    def calculate_reward(self, action: int) -> float:
        """
        An example reward function. This is the one function that users will likely
        wish to inject their own creativity into.

        Warning!
        This is function is a showcase of functionality designed to show as many possible
        environment control features as possible. It is also designed to run quickly
        on small computers. This is a benchmark, it is *not* for live production.

        :param action: int = The action made by the agent for the current candle.
        :return:
        float = the reward to give to the agent for current step (used for optimization
            of weights in NN)
        """

In [ ]:
    def _update_unrealized_total_profit(self):
        """
        Update the unrealized total profit in case of episode end.
        """
        if self._position in (Positions.Long, Positions.Short):
            pnl = self.get_unrealized_profit()
            if self.compound_trades:
                # assumes unit stake and compounding
                unrl_profit = self._total_profit * (1 + pnl)
            else:
                # assumes unit stake and no compounding
                unrl_profit = self._total_profit + pnl
            self._total_unrealized_profit = unrl_profit

    def _update_total_profit(self):
        pnl = self.get_unrealized_profit()
        if self.compound_trades:
            # assumes unit stake and compounding
            self._total_profit = self._total_profit * (1 + pnl)
        else:
            # assumes unit stake and no compounding
            self._total_profit += pnl

    def current_price(self) -> float:
        return self.prices.iloc[self._current_tick].open

    def get_actions(self) -> Type[Enum]:
        """
        Used by SubprocVecEnv to get actions from
        initialized env for tensorboard callback
        """
        return self.actions

In [ ]:
    # Keeping around in case we want to start building more complex environment
    # templates in the future.
    def most_recent_return(self):
        """
        Calculate the tick to tick return if in a trade.
        Return is generated from rising prices in Long
        and falling prices in Short positions.
        The actions Sell/Buy or Hold during a Long position trigger the sell/buy-fee.
        """
        # Long positions
        if self._position == Positions.Long:
            current_price = self.prices.iloc[self._current_tick].open
            previous_price = self.prices.iloc[self._current_tick - 1].open

            if (self._position_history[self._current_tick - 1] == Positions.Short
                    or self._position_history[self._current_tick - 1] == Positions.Neutral):
                previous_price = self.add_entry_fee(previous_price)

            return np.log(current_price) - np.log(previous_price)

        # Short positions
        if self._position == Positions.Short:
            current_price = self.prices.iloc[self._current_tick].open
            previous_price = self.prices.iloc[self._current_tick - 1].open
            if (self._position_history[self._current_tick - 1] == Positions.Long
                    or self._position_history[self._current_tick - 1] == Positions.Neutral):
                previous_price = self.add_exit_fee(previous_price)

            return np.log(previous_price) - np.log(current_price)

        return 0


O método most_recent_return calcula o retorno mais recente com base no movimento de preços entre ticks consecutivos, levando em consideração a posição do agente (Long ou Short) e aplicando taxas de entrada ou saída quando necessário. Esse retorno é calculado como um retorno logarítmico, que é uma forma comum de representar mudanças percentuais em aprendizado por reforço financeiro.

Detalhamento do Método most_recent_return
Contexto de Posições:

O método considera dois tipos de posições:
Long (compra): O agente ganha com a valorização do preço.
Short (venda): O agente ganha com a queda do preço.
Quando o agente não está em uma posição (Neutral), o retorno é 0, pois não há variação acumulada.
Cálculo para Posições Long:

Se a posição atual (self._position) é Long:
current_price e previous_price são os preços de abertura (open) nos ticks atual e anterior.
Taxa de Entrada: Se a posição anterior era Short ou Neutral, o agente está iniciando uma nova posição Long, e uma taxa de entrada é aplicada ao preço anterior (previous_price = self.add_entry_fee(previous_price)).
O retorno é calculado como a diferença logarítmica entre current_price e previous_price, ou seja: \text{return} = \log(\text{current_price}) - \log(\text{previous_price})
Cálculo para Posições Short:

Se a posição atual é Short:
current_price e previous_price são obtidos da mesma forma.
Taxa de Saída: Se a posição anterior era Long ou Neutral, o agente está iniciando uma nova posição Short, e uma taxa de saída é aplicada ao preço anterior (previous_price = self.add_exit_fee(previous_price)).
O retorno é então a diferença logarítmica invertida: \text{return} = \log(\text{previous_price}) - \log(\text{current_price})
Esse cálculo reflete o ganho do agente em uma posição Short, onde ele se beneficia da queda de preços.
Retorno para Posições Neutras:

Se o agente está em posição Neutral (nenhuma posição aberta), o método retorna 0, indicando que não há lucro ou perda quando o agente não está em uma posição de negociação.
Observações Adicionais
Retorno Logarítmico: A utilização do logaritmo natural para calcular o retorno é comum, pois ele fornece uma maneira aditiva de acumular retornos ao longo do tempo, o que facilita a análise e o treinamento do agente.
Taxas de Entrada e Saída: Essas taxas são aplicadas ao preço inicial ao entrar em uma nova posição, simulando custos de transação. Isso torna o ambiente mais realista, pois o agente deve considerar os custos de negociação ao avaliar suas decisões.
Exemplo de Uso
Esse método pode ser usado a cada tick para calcular o retorno mais recente da posição atual do agente. Ele fornece uma métrica que o agente pode utilizar para atualizar sua recompensa ou avaliar o sucesso de uma ação tomada.

Resumo
O método most_recent_return é uma ferramenta importante para calcular os ganhos (ou perdas) mais recentes do agente, levando em consideração a posição (Long ou Short) e aplicando taxas de transação. Ele permite que o agente avalie o impacto de cada decisão de negociação, um aspecto essencial para o treinamento de um modelo de aprendizado por reforço em ambientes financeiros.

In [ ]:
def update_portfolio_log_returns(self, action):
    self.portfolio_log_returns[self._current_tick] = self.most_recent_return(
        action)